In [1]:

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import signal

In [122]:
path_file = "D:\Edu\Lab\Bin_Classifier_Project\data/total_data_v4.csv"   # 41649_result_data_v1

df = pd.read_csv(path_file)
df.head()

,D_ID,Ch,Y,Length,Rate,0,1,2,3,4,...,502,503,504,505,506,507,508,509,510,511
0,41226,ch3,0.0,0.021,4.0,0.796870,0.805668,0.813404,0.820079,0.825693,...,-0.546500,-0.528759,-0.505209,-0.475852,-0.440686,-0.399711,-0.352928,-0.300337,-0.241938,-0.17773
1,41226,ch3,0.0,0.025,4.0,0.384770,0.361413,0.341766,0.325828,0.313599,...,0.345798,0.364561,0.387407,0.405740,0.418579,0.425926,0.427781,0.424143,0.415013,0.40039
2,41226,ch3,0.0,0.023,4.0,0.169920,0.174718,0.178798,0.182162,0.184809,...,-0.401508,-0.429319,-0.453263,-0.471913,-0.485271,-0.493335,-0.496106,-0.493584,-0.485769,-0.47266
3,41226,ch3,0.0,0.021,4.0,0.476560,0.499920,0.519142,0.534226,0.545171,...,0.839651,0.860380,0.873599,0.879308,0.877508,0.868198,0.851378,0.827048,0.795209,0.75586
4,41226,ch3,0.0,0.025,4.0,0.021484,0.025230,0.036639,0.055710,0.082443,...,0.352659,0.362767,0.373820,0.380639,0.382823,0.380373,0.373287,0.361566,0.345211,0.32422


In [123]:
stats_df = pd.DataFrame(np.column_stack([df.Y, df.Length]), columns=["Y", "Length"])
signal_fragments = df.loc[:, df.columns[5:]].to_numpy()
stats_df.shape

(702, 2)

In [124]:
def find_max_num(arr: np.array, arr2: np.array = None):
    f_increase = False
    max_count = 0
    max_v = 0
    maximums = []

    for i in range(1, arr.shape[0]):
        if not f_increase and arr[i - 1] < arr[i]:
            f_increase = True
            max_v = arr[i]
            # print(arr[i - 1], arr[i])
        if f_increase:
            max_v = max(arr[i], max_v)
            if arr[i - 1] > arr[i] and abs(max_v - arr[i]) / max_v > 0.3:
                f_increase = False
                # print(max_v, arr[i], abs(max_v - arr[i]) / max_v, "\n")
                maximums.append(max_v)
                max_count += 1
    if len(maximums) > 0:
        if arr2 is not None:
            max_max_freq = arr2[arr == max(maximums)][0]
        else:
            max_max_freq = -1.0
        if len(maximums) > 1:
            maximums = np.array(maximums)
            mean_max_ratio = (maximums[maximums != maximums.max()] / maximums.max()).max()
        else:
            mean_max_ratio = 1.0
    else:
        return [0.0, 0.0, -1.0]

    return [max_count, mean_max_ratio, max_max_freq]


def down_to_zero(x, edge=0.05):
    return x if x > edge else 0.0


def fragment_fft(arr: np.array):
    fft = np.fft.fft(arr)
    fft_v = fft.real ** 2 + fft.imag ** 2

    filter_values = np.vectorize(lambda x: down_to_zero(x, edge=fft_v.max() * 0.2))
    fft_v_filter = filter_values(fft_v)

    frequency = np.unique(np.abs(np.fft.fftfreq(arr.shape[0])))
    fft_v_filter = fft_v_filter[:frequency.shape[0]]

    return [frequency, fft_v_filter]

In [125]:
fft_stats = []
for i in range(signal_fragments.shape[0]):
    signal_data = signal_fragments[i]
    max_point, min_point = signal_data.max(), signal_data.min()
    # normalise all values
    # signal_data = (signal_data - (max_point + min_point) / 2) / (max_point - min_point)

    n_diff = 1
    signal_data_d1 = np.diff(signal_data, n=n_diff)

    b, a = signal.butter(5, 0.1)
    signal_data_f = signal.filtfilt(b, a, signal_data_d1)

    fft = np.fft.fft(signal_data_f)
    fft_v = fft.real ** 2 + fft.imag ** 2

    filter_values = np.vectorize(lambda x: down_to_zero(x, edge=fft_v.max() * 0.05))
    fft_v_filter = filter_values(fft_v)  # fft_v

    frequency = np.unique(np.abs(np.fft.fftfreq(signal_data_f.shape[0])))
    fft_v_filter = fft_v_filter[:frequency.shape[0]]

    # print(fft_v[(frequency > 0) & (frequency < 0.1)])
    fft_stats.append(find_max_num(fft_v_filter, frequency))

fft_stats = np.array(fft_stats)
# fft_stats = np.column_stack([fft_stats, signal_fragments])
fft_stats.shape

(702, 3)

In [126]:
# def get_fft_stats(fragment: np.array):
#     fragment_d1 = np.diff(fragment, n=1)
#     b, a = signal.butter(5, 0.1)
#
#     filtered_fragment = signal.filtfilt(b, a, fragment_d1)
#     freq, fft = fragment_fft(filtered_fragment)
#     return np.array(find_max_num(freq, fft))
#
# def fragments_d1(fragment: np.array):
#     # print(fragment)
#     return np.diff(fragment, n=1)
#
# # signal_fragments_d1 = np.apply_along_axis(fragments_d1, 1, signal_fragments)
#
# fft_stats = np.apply_along_axis(get_fft_stats, 1, signal_fragments)
# fft_stats.shape

In [127]:
stats_df[["max_count", "mean_max_ratio", "max_freq"]] = fft_stats

print(stats_df.shape)
stats_df.head()

(702, 5)


,Y,Length,max_count,mean_max_ratio,max_freq
0,0.0,0.021,7.0,0.774922,0.011742
1,0.0,0.025,6.0,0.557506,0.023483
2,0.0,0.023,7.0,0.362513,0.015656
3,0.0,0.021,7.0,0.786896,0.011742
4,0.0,0.025,7.0,0.719183,0.021526


In [128]:
print(stats_df.groupby(["Y", "max_count"]).agg(["count", "mean"]))  # [["mean_max_ratio", "max_freq"]]

              Length           mean_max_ratio           max_freq          
               count      mean          count      mean    count      mean
Y   max_count                                                             
0.0 3.0            3  0.022000              3  0.519013        3  0.020874
    4.0           22  0.019318             22  0.580734       22  0.022861
    5.0           59  0.021068             59  0.669038       59  0.022986
    6.0           94  0.021968             94  0.683773       94  0.022442
    7.0          110  0.022709            110  0.679460      110  0.024337
    8.0           59  0.021898             59  0.755692       59  0.022090
    9.0           19  0.023947             19  0.747414       19  0.028839
    10.0           2  0.023500              2  0.972209        2  0.022505
    11.0           1  0.025000              1  0.701950        1  0.015656
1.0 1.0            1  0.021000              1  1.000000        1  0.025440
    2.0            4  0.0

In [129]:
def plot_signal_fft(log_df, fig, axs):
    x_ = np.arange(0, 512, 1)

    for col in log_df.columns[1:]:
        y_ = log_df[col]

        fft = np.fft.fft(y_)
        fft_v = fft.real ** 2 + fft.imag ** 2

        # filter_values = np.vectorize(lambda x: down_to_zero(x, edge=fft_v.max() * 0.2))
        fft_v_filter = fft_v  # filter_values(fft_v)

        frequency = np.unique(np.abs(np.fft.fftfreq(y_.shape[0])))
        fft_v_filter = fft_v_filter[:frequency.shape[0]]
        m, r, f = find_max_num(fft_v_filter)
        axs[0].plot(frequency, fft_v_filter, label=f"{col}. C_max: {m}, Rat: {round(r, 2)}, Fr: {round(f, 2)}")
        axs[1].plot(x_, y_, label=col)

    axs[0].set_title(f'FFT', fontdict={'fontsize': 10}, loc="left")
    axs[0].set_xlabel('Freq [Hz]')
    axs[0].grid(True)
    axs[0].set_ylim([0, 5])
    axs[0].legend(loc='upper right', borderaxespad=0.)  # bbox_to_anchor=(0.005, 1),

    axs[1].set_title(f"Signal", fontdict={'fontsize': 10}, loc="left")
    axs[1].set_xlabel('Time [milliseconds]')
    axs[1].legend(bbox_to_anchor=(1.005, 1), loc='upper left', borderaxespad=0.)
    x_min = log_df["t"].min()
    x_max = log_df["t"].max()
    try:
        axs[1].set_xticks(np.round(np.arange(x_min, x_max + 1e-3, (x_max - x_min) / 8), 5))
    except ValueError as e:
        pass
    axs[1].grid(True)

In [130]:
marks = stats_df["Y"].to_numpy()
max_count = stats_df["max_count"].to_numpy()

fil_ids = np.where((marks == 1) & (max_count > 2))[0]
print(fil_ids, fil_ids.shape[0])

[  5  14  22  25  26  32  38  39  44  45  47  50  52  53  57  58  59  60
  62  64  65  68  72  73  74  75  82  85  86  92  93  97  98  99 100 101
 104 105 108 112 113 119 120 122 123 126 128 129 130 131 136 149 151 157
 163 168 170 171 177 178 181 183 184 190 192 194 195 197 199 202 204 206
 210 213 214 215 216 217 222 223 224 226 229 236 237 239 240 242 243 245
 247 248 249 250 253 255 260 265 266 268 269 270 272 274 275 277 279 282
 283 287 288 290 291 292 293 294 295 296 297 299 300 302 305 307 308 309
 310 313 316 318 319 321 322 324 327 329 331 332 335 341 343 345 347 350
 351 353 355 356 358 359 360 365 369 372 374 376 378 379 381 383 386 387
 388 390 392 396 397 398 401 403 404 405 406 408 413 416 418 419 420 424
 426 427 429 431 432 433 435 436 439 441 442 447 448 449 451 454 455 456
 457 461 464 465 470 472 473 474 475 477 478 479 483 486 487 488 489 492
 494 495 497 498 499 501 502 504 505 507 509 510 511 512 513 514 515 517
 518 523 527 528 530 531 533 535 536 538 540 543 54

In [131]:
for i in range(min(fil_ids.shape[0], 10)):
    signal_data = signal_fragments[fil_ids[i]]

    n_diff = 1
    signal_data_d1 = np.diff(signal_data, n=n_diff)

    log_df = pd.DataFrame(np.column_stack([np.arange(0, 512, 1), signal_data]), columns=["t", "ch11"])
    log_df["ch11"] = signal_data
    # if log_df is not None:
    #     log_df[f"ch11_{n_diff}d"] = np.concatenate([np.diff(signal_data, n=n_diff), [0]*n_diff])

    # sos = signal.butter(5, 0.1, output="sos")
    #
    # signal_data_d1 = signal.sosfilt(sos, signal_data_d1)

    b, a = signal.butter(5, 0.1)

    signal_data_d1 = signal.filtfilt(b, a, signal_data_d1)
    log_df["ch11_b"] = np.concatenate([signal_data_d1, [0] * n_diff])

    m = signal_data_d1.mean()
    std = signal_data_d1.std()
    log_df["ch11_mn"] = np.array([m] * (signal_data_d1.shape[0]+1))
    log_df["ch11_std"] = np.array([m + std] * (signal_data_d1.shape[0]+1))
    log_df["ch11_-std"] = np.array([m - std] * (signal_data_d1.shape[0]+1))

    fig, axs = plt.subplots(1, 2, figsize=(8, 4))
    # plt.tight_layout()
    plot_signal_fft(log_df, fig, axs)
    plt.savefig(f"img/1/fragment_41226_{fil_ids[i]}.png")
    plt.close()